In [ ]:
from itertools import chain
from pathlib import Path
from google_photos_takeout_model.models.albums import Album
from google_photos_takeout_model.models.media_items import get_media_items
from devtools import pprint

In [ ]:
albums = Path("D:/Google takeout/2024-11-20-photos")
people = list((albums / "people").iterdir())
undated = list((albums / "undated").iterdir())
trash = albums / "trash"
print(
    f"{len(people)=}",
    f"{len(undated)=}",
    f"Trash: {trash.relative_to(albums).as_posix()}",
    sep="\n",
)

In [ ]:
all_years = albums / "years"
years = {year.name: get_media_items(year) for year in all_years.iterdir()}
all_media_items = list(chain.from_iterable(years.values()))
print(
    f"{len(years)=}",
    f"{len(all_media_items)=}",
    sep="\n",
)

In [ ]:
pprint(all_media_items[-1])

In [ ]:
dated = {
    album_path.name: Album.from_path(album_path)
    for album_path in [
        *(albums / "dated").iterdir(),
        *(albums / "dated-done").iterdir(),
    ]
}
print(
    f"{len(dated)=}",
    sep="\n",
)

In [ ]:
pprint(next(iter(dated.values())).media_items[-1])

In [ ]:
# class Album(BaseModel):
#     path: Path

#     @property
#     def metadata(self) -> Path:
#         return self.path / "metadata.json"

#     @property
#     def file_metadata(self) -> list[Path]:
#         return list(self.path.glob("[!metadata]*.json"))

#     @property
#     def files(self) -> list[Path]:
#         return list(self.path.glob("[!metadata]*[!.json]"))

#     @property
#     def missing(self) -> list[Path]:
#         return [
#             path.with_name(path.stem)
#             for path in self.file_metadata
#             if not path.with_name(path.stem).exists()
#         ]
album = dated["2008-10-25 Pool"]
found_media_items_metadata_names = [
    media_item.metadata_path.name for media_item in album.media_items
]
all_media_items_metadata = list(album.path.glob("[!metadata]*.json"))
it = iter(dated.values())

In [ ]:
album = next(it)
found_media_items_metadata_names = [
    media_item.metadata_path.name for media_item in album.media_items
]
all_media_items_metadata = list(album.path.glob("[!metadata]*.json"))
metadata_paths_with_missing_media_items = [
    metadata_path
    for metadata_path in all_media_items_metadata
    if metadata_path.name not in found_media_items_metadata_names
]
metadata_paths_with_missing_media_items
# [
#     photo.relative_to(all_years).as_posix()
#     for photo in all_media_items
#     if photo.name in missing_names
# ]